In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
from scipy.ndimage import gaussian_filter1d
from scipy.signal import convolve
from brian2 import *

In [3]:
sys.path.append("../lib") # add code library to paths

# add lib to path 
from model_beta import Model



In [4]:
# TODO: turn this into  script

In [5]:
netname = 'recurrent_nn'
model = Model(netname)  # initiate the model


../output/recurrent_nn


In [8]:
params = {
'sig_c' : 0.05  ,    # bipolar receptive fied size ~ 1/5 of the actual size [mm]
'tau1' : 0.04   ,    # excitation time constant of the temporal filter [s]
'tau2' : 0.0876 ,    # rebound time constant [s]
'bifw' : 0.     ,    # relative weight of rebound/excitation [1]

'scale_mV' : 20.,     # UPDATED 

'tauA' : 0.15   ,     # time constant of amacrine cells [s]
'tauB' : 0.08   ,     # time constant of bipolar cells [s]
'tauG' : 0.01   ,     # time contant of ganglion cells [s]

'wAB' : 10.     ,     # synaptic weight from bipolar to amacrine [Hz]
'wBA' : -10.    ,     # synaptic weight from amacrine to bipolar [Hz]
'wGB' : 4.0     ,     # synaptic weight from bipolar to gangion  [Hz].   #UPDATED
'wGA' : 0.      ,     # synaptic weight from amacrine  to gangion  [Hz]

'slope' : 5     ,     # slope for ganglion cell recritifation [Hz/mV].   #UPDATED
'threshold' : 0 ,     # threshold for ganglion cell recritifation [Hz/mV]
'sig_pool' : 0.065 ,  # sigma for gaussian pooling in ganlion gells [mm]


'spacing' : 0.005  ,  # spacing of cells on the lattice [mm]
'dt' : 0.001      ,   # integration time step [s]
'N' : 512          ,  # number of neurons in each layer [1]


# idx = int(N/2)
# b =  0.160         # half bar width [mm]
# speed =  0.8       # speed [mm/s]  
}


In [9]:

# create connectivity matrix TODO write a class for this !!!

C = np.zeros((params['N'],params['N']))
ii = np.arange(0,params['N']-1,1).astype(int)  # list of source cells for rightward connections
ji = ii + 1                          # target cells of rightward connections


C[ii,ji] = 1

ii = np.arange(1,params['N'],1).astype(int)    # list of source cells for leftward connections
ji = ii - 1                         # target cells of leftward connections

C[ii,ji] = 1


In [10]:

model.set_params(params)
model.set_connectivity(C)


# simulate response to global impulse

In [11]:

dur = 3.                      # duration of the simulation [s]
tps = int(dur/params['dt'])             # number of time-points in the simulation [1]
time = np.arange(0,tps)*params['dt']    # time array for the simulation 1xtps, in [s]

# initialize the stimulus
S = np.zeros((params['N'],tps))
S[:,int(tps/2)] = 1


In [12]:
model.set_stimulus(S,'gImpulse')
model.simulate()

../output/recurrent_nn/gImpulse


# simulate response to local Impulse

In [17]:

dur = 3.                      # duration of the simulation [s]
tps = int(dur/params['dt'])             # number of time-points in the simulation [1]
time = np.arange(0,tps)*params['dt']    # time array for the simulation 1xtps, in [s]

# initialize the stimulus
S = np.zeros((params['N'],tps))
S[206,int(tps/2)] = 1/params['dt']*0.05


In [18]:
model.set_stimulus(S,'lImpulse')
model.simulate()

../output/recurrent_nn/lImpulse


In [19]:
params['spacing']

0.005